In [1]:
import json
import networkx as nx
#from networkx import graphviz_layout
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.cm as cm

from sklearn.preprocessing import normalize
from matplotlib.widgets import Button
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
from matplotlib.patches import Wedge


import sys
from PyQt5.QtWidgets import QDialog, QApplication, QPushButton, QVBoxLayout, QLabel, QComboBox, QStyleFactory, \
    QGridLayout, QSlider
import pandas as pd
import re
import pyqtgraph as pg
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from PyQt5.QtCore    import Qt
from PyQt5.uic.properties import QtWidgets, QtCore

from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
import random
from matplotlib.colors import Normalize

from matplotlib.figure import Figure
import math

from sklearn.preprocessing import normalize
from anytree import Node, RenderTree, AsciiStyle, PreOrderIter, PostOrderIter
from anytree.importer import DictImporter

In [2]:
with open('flare.json') as f:
    js_data = json.loads(f.read())

In [ ]:
#create graph
importer = DictImporter()
root = importer.import_(js_data )

children_curr = [root]
cols = {}
for j in children_curr:
    #print('y',j)
    c = 0
    for i in PostOrderIter(j):
        try :
            c += i.value
        except AttributeError:
            continue
    cols[j.name] = c
    try:
        #print((list(j.children)))
        children_curr+=(list(j.children))
    except:
        continue

node_lis = []
edge_lis = []
col_lis = []


#adds nodes vals and edge vals to lists for network
def parse_json(parent, data):
    curr = parent
    if curr != None:
        edge_lis.append((parent, data['name']))
   
    col_lis.append(cols[data['name']])
    node_lis.append(data['name'])
    if 'children' in data:
        for i in data['children']:
            parse_json(data['name'],i)
        
    else:
        return
  
parse_json(None, js_data)


G = nx.DiGraph()
G.add_nodes_from(node_lis)
G.add_edges_from(edge_lis)



sh = nx.shortest_path_length(G, 'flare')
#pos=nx.nx_agraph.graphviz_layout(G,prog='twopi')
#nx.draw(G,pos,node_size=20,alpha=0.5,with_labels=True)

In [4]:
class Window(QDialog):


    # constructor
    def __init__(self, parent=None):
        super(Window, self).__init__(parent)
        title = 'plot'
        width = 900
        height = 400
        left = 900
        top = 400
        self.x = ""
        self.y = 0
        c = 0
        s= 0
        #self.setGeometry(400, 400, 900, 900)
        
        
        self.fig = plt.figure()
        self.canvas = FigureCanvas(self.fig)
        
        
        self.button = QPushButton("Change", self)
        self.button.setCheckable(True) 
        self.button.pressed.connect(self.find)
        
        grid = QVBoxLayout()
        #grid.addWidget(self.canvas)
      
        self.mySlider = QSlider(Qt.Horizontal, self)
        self.mySlider.valueChanged[int].connect(self.changeValue)
        self.mySlider.setRange(0, 4)
        

        grid.addWidget(self.mySlider)
        grid.addWidget(self.button)
        self.setLayout(grid)
        self.show()
        self.update()
        
        #x, y, c, s = self.find()

        
    def find(self):

            self.fig.clear()
            plt.clf()
            #ax1 = self.fig.add_subplot(111)
            if self.button.isChecked():
                #ax1.scatter(x=[8,3],y=[3,0])
                p = 'twopi'
                args = ''
            else:
                p = 'dot'
                args = "-Grankdir=LR"
            pos=nx.nx_agraph.graphviz_layout(G,prog=p, args = args)
            nx.draw(G,pos,node_size=20,alpha=0.5,node_color=col_lis, with_labels=True)
            
          
            def update_annot(ind):
                node = ind["ind"][0]
                pos = pos[list(G.nodes)[node]]
                annot.xy = pos
                node_attr = {'node': node}
                
                text = 'HI'
                annot.set_text(text)

            def hover(event):
                vis = annot.get_visible()
                if event.inaxes == ax:
                    cont_nodes, ind_nodes = nodes.contains(event)
                    if cont_nodes:
                        update_annot(ind_nodes)
                        annot.set_visible(True)
                        fig.canvas.draw_idle()
                    else:
                        if vis:
                            annot.set_visible(False)
                            fig.canvas.draw_idle()
         
            self.canvas.mpl_connect("motion_notify_event", hover)

            
            
            c = plt.get_cmap("viridis")
            m = ScalarMappable(Normalize(min(col_lis), max(col_lis)), c)

            clb = plt.colorbar(m)

            #plt.show()
            self.canvas.draw()
            
    def changeValue(self, value):

        self.fig.clear()
        plt.clf()
        key_sub = [k for k,v in sh.items() if v > value]
       

        G.remove_nodes_from(key_sub)
        # create an axis
        #ax = self.fig.add_subplot(111)
        if self.button.isChecked():
                #ax1.scatter(x=[8,3],y=[3,0])
                p = 'twopi'
                args=""
        else:
                p = 'dot'
                args = "-Grankdir=LR"
        sub_col = []
        for i in G.nodes:
            sub_col.append(cols[i])
        pos=nx.nx_agraph.graphviz_layout(G,prog=p, args=args)
        nx.draw(G,pos,node_size=20,node_color=sub_col,alpha=0.5,with_labels=True)
        G.add_nodes_from(key_sub)
        G.add_edges_from(edge_lis)
        c = plt.get_cmap("viridis")
        m = ScalarMappable(Normalize(min(col_lis), max(col_lis)), c)
        clb = plt.colorbar(m)

        plt.show()
        
        self.canvas.draw()



if __name__ == '__main__':
        # creating apyqt5 application
        app = QApplication(sys.argv)

        # creating a window object
        main = Window()

        # showing the window
        main.show()

        # loop
        sys.exit(app.exec_())

SystemExit: 0

/Users/kruthikrishnappa/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3333: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
